In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('reddit-university-data.csv')

In [3]:
df.head()

,id,university,title,content,comments,author,created_utc,score
0,1fylsog,harvard,"Register to vote by October 26th, and vote in ...","On November 5th, Massachusetts will vote not j...",[],EllieDai,2024-10-08 04:54:41,6
1,1g0gvaa,harvard,Has the Harvard Crimson ever retracted/apologi...,"This is a very specific question, and it obvio...",['College kids hold misguided views. More news...,Ouroboros963,2024-10-10 17:15:02,22
2,1g0dbjv,harvard,Harvard Alumnus Wins Chemistry Nobel,NaN,"['For contributors to AI?', ""I don't think he ...",ThrillSurgeon,2024-10-10 12:54:50,10
3,1g0k72t,harvard,Americans Victor Ambros of UMass Chan and Gary...,NaN,[],mlockerottinghaus,2024-10-10 20:02:25,1
4,1g09rjw,harvard,History of Science,History of science majors/those of you familia...,"[""I'm a graduate student so can't answer your ...",Pretzel2192,2024-10-10 08:52:19,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           800 non-null    object
 1   university   800 non-null    object
 2   title        800 non-null    object
 3   content      726 non-null    object
 4   comments     800 non-null    object
 5   author       800 non-null    object
 6   created_utc  800 non-null    object
 7   score        800 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.1+ KB


In [5]:
df = df.dropna(subset=['content'])

In [11]:
df = df[df['comments'].apply(lambda x: x != '[]')]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 608 entries, 1 to 798
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           608 non-null    object
 1   university   608 non-null    object
 2   title        608 non-null    object
 3   content      608 non-null    object
 4   comments     608 non-null    object
 5   author       608 non-null    object
 6   created_utc  608 non-null    object
 7   score        608 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 42.8+ KB


In [14]:
df.head()

,id,university,title,content,comments,author,created_utc,score
1,1g0gvaa,harvard,Has the Harvard Crimson ever retracted/apologi...,"This is a very specific question, and it obvio...",['College kids hold misguided views. More news...,Ouroboros963,2024-10-10 17:15:02,22
4,1g09rjw,harvard,History of Science,History of science majors/those of you familia...,"[""I'm a graduate student so can't answer your ...",Pretzel2192,2024-10-10 08:52:19,5
7,1g07l8g,harvard,Is this a Harvard startup?,I’ve been seeing a website called Shout! being...,['It seems to be a B.U. students startup:\n\n[...,Bitter-Air-8473,2024-10-10 06:53:32,0
8,1fzbmh1,harvard,Scenic Coastal/Lake Areas Near Harvard,My 25th reunion is coming up and I want to gat...,['Take the ferry from Boston harbor to Provinc...,wheelwatcher222,2024-10-09 03:19:35,7
9,1fzl93q,harvard,Historical Tour of Harvard,My family is coming to visit next month and I ...,['It was up and running as of a week or so ago...,Spare_Pay_2944,2024-10-09 12:09:23,1
